In [2]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np

pd.set_option('display.max_colwidth', None)



In [3]:
def set_environment_variables_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if line and not line.startswith('#'):  # Ignore empty lines and comments
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()
file_path = '../../.env.local'
set_environment_variables_from_file(file_path)
openai.api_key = os.getenv('OPENAI_KEY')

In [4]:
def estimate_price(n):
    num_tokens = 900 * n
    # per thousend tokens 0.0001 $
    price_per_thousend = 0.0001
    price = round(num_tokens/1000 * price_per_thousend, 5)

    response = input(f"Estimated price {price} $\nProceed? (y/n): ")
    if response.lower() == 'y':
        print("Price accepted...")
        # Continue with your notebook code here

    elif response.lower() == 'n':
        print("Price declined, interrupting...")
        raise KeyboardInterrupt  # Raises a KeyboardInterrupt to stop execution

    else:
        print("Invalid input. interrupting...")
        raise KeyboardInterrupt
        # Prompt the user again or take appropriate action

In [5]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [6]:
basepath = '../../datasets/preprocessed_datasets/retailrocket/'
product_data = pd.read_pickle(basepath +'item_data_extracted.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=1000, random_state=2).reset_index(drop=True)
display(product_data)

,itemid,categoryid,available,properties,property_values
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189, 202, 225, 227, 283, 364, 42, 561, 6, 678, 698, 764, 776, 790, 839, 869, 888, 917]","[1276750, n3.168 1144008, n15360.000 628176 n12288.000, 679677, 1168476, n4800.000 270060 924073, 519769, n96.000 1206660, 708480, 478989, 1301326, 1152934 1238769, 66094 372274 478989, 20716, n204.000, 1294803 101489, [66094, 66094, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769], 372274, 1152934 1238769, 1285872, 318611, n91200.000, 372274, 769062, 478989, 478989]"
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 283, 284, 296, 33, 364, 438, 59, 6, 606, 664, 678, 689, 698, 730, 764, 768, 776, 790, 813, 839, 888, 917, 93, 981]","[769062, 1154859, 679677, 519769, 769062, 172646, 1020281 150169 1263524, 769062, 769062, 513325 1020281 1204938 172646 72261 30603 898004 140719 302848 1204938 190728 532620 572715 1128577 162103 1097148 288731 418473 150169 855390 836649 113294 435459 1103229 1019257 1272915 707081 13541 1161933 610340 804917, 769062, 866110, 1128577 1000087 421694, 1299107, 807503 1210748, 769062, 1020281, 769062, 992329, 1204938, 150169 1263524, 1020281, 769062, 1285872, n36.000, 379040, [n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6000.000, n6000.000, n6240.000, n5760.000, n5760.000, n5760.000, n5760.000, n5760.000], 814966, 1204938, [172646 1154859, 172646 1154859, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646], n372361069212.000, 769062, 769062]"
2,2,1305,0,"[1063, 112, 159, 202, 282, 283, 318, 332, 364, 443, 641, 678, 698, 699, 764, 776, 778, 790, 839, 877, 888, 917]","[n396.000 145688, 679677, 519769, 648485 n6000.000 n72.000, n192.000 145688, 822092 325894 504272 147366 343631 648485 n6000.000 n72.000 504272 147366 343631 648485 n6000.000 639502 n72.000 936990 949802 1113810 504272 653611 325894 1262898 796845 161652 150169 353870 1309799 1030323 608362 680618 1128577 636684 1081984 703408, 822092 325894, n72.000, 1260440, n6000.000 1144008, n288.000 145688, 147366 343631, 822092 325894 504272, n14616.000 10317, 1285872, 1005623, n840.000 1148727 237874, n41040.000, 147366 343631, n32.400 396934, 648485 n6000.000 n72.000 n72.000 468140, 648485 n6000.000 639502 n72.000]"
3,3,1171,0,"[1025, 1080, 112, 159, 202, 227, 250, 283, 30, 326, 33, 364, 453, 459, 562, 6, 604, 658, 678, 689, 698, 764, 776, 790, 839, 888, 917]","[769062, 769062, 679677, 519769, 261419, 1305767, 300154, [1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419], 769062, 769062, 279887 421694, 354388, 769062, 769062, 769062, 1305767, 769062, 769062, 327918, 150169 1182824, 1305767, 1285872, 284910, n1560.000, 327918, 261419, 261419 829653]"
4,4,1038,0,"[112, 115, 159, 202, 227, 28, 283, 33, 364, 591, 6, 678, 689, 698, 764, 776, 790, 839, 841, 888, 897, 917, 981]","[679677, n24.000, 519769, 371058 71429, 588652 1091491, 150169 176547, 588652 768453 508476 371058 71429, 1128577 1000087 421694, 1142657, 1116693, 588652 1091491, 508476, 371058 150169 176547, 588652 10914

In [7]:
product_data['merged_properties'] = [
    {key: value for key, value in zip(row['properties'], row['property_values'])}
    for _, row in product_data.iterrows()
]

# Output the resulting dictionary
display(product_data)


,itemid,categoryid,available,properties,property_values,merged_properties
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189, 202, 225, 227, 283, 364, 42, 561, 6, 678, 698, 764, 776, 790, 839, 869, 888, 917]","[1276750, n3.168 1144008, n15360.000 628176 n12288.000, 679677, 1168476, n4800.000 270060 924073, 519769, n96.000 1206660, 708480, 478989, 1301326, 1152934 1238769, 66094 372274 478989, 20716, n204.000, 1294803 101489, [66094, 66094, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769], 372274, 1152934 1238769, 1285872, 318611, n91200.000, 372274, 769062, 478989, 478989]","{'1036': '1276750', '1056': 'n3.168 1144008', '11': 'n15360.000 628176 n12288.000', '112': '679677', '127': '1168476', '139': 'n4800.000 270060 924073', '159': '519769', '177': 'n96.000 1206660', '189': '708480', '202': '478989', '225': '1301326', '227': '1152934 1238769', '283': '66094 372274 478989', '364': '20716', '42': 'n204.000', '561': '1294803 101489', '6': ['66094', '66094', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769'], '678': '372274', '698': '1152934 1238769', '764': '1285872', '776': '318611', '790': 'n91200.000', '839': '372274', '869': '769062', '888': '478989', '917': '478989'}"
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 283, 284, 296, 33, 364, 438, 59, 6, 606, 664, 678, 689, 698, 730, 764, 768, 776, 790, 813, 839, 888, 917, 93, 981]","[769062, 1154859, 679677, 519769, 769062, 172646, 1020281 150169 1263524, 769062, 769062, 513325 1020281 1204938 172646 72261 30603 898004 140719 302848 1204938 190728 532620 572715 1128577 162103 1097148 288731 418473 150169 855390 836649 113294 435459 1103229 1019257 1272915 707081 13541 1161933 610340 804917, 769062, 866110, 1128577 1000087 421694, 1299107, 807503 1210748, 769062, 1020281, 769062, 992329, 1204938, 150169 1263524, 1020281, 769062, 1285872, n36.000, 379040, [n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6000.000, n6000.000, n6240.000, n5760.000, n5760.000, n5760.000, n5760.000, n5760.000], 814966, 1204938, [172646 1154859, 172646 1154859, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646], n372361069212.000, 769062, 769062]","{'0': '769062', '1036': '1154859', '112': '679677', '159': '519769', '185': '769062', '202': '172646', '227': '1020281 150169 1263524', '238': '769062', '280': '769062', '283': '513325 1020281 1204938 172646 72261 30603 898004 140719 302848 1204938 190728 532620 572715 1128577 162103 1097148 288731 418473 150169 855390 836649 113294 435459 1103229 1019257 1272915 707081 13541 1161933 610340 804917', '284': '769062', '296': '866110', '33': '1128577 1000087 421694', '364': '1299107', '438': '807503 1210748', '59': '769062', '6': '1020281', '606': '769062', '664': '992329', '678': '1204938', '689': '150169 1263524', '698': '1020281', '730': '769062', '764': '1285872', '768': 'n36.000', '776': '379040', '790': ['n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6000.000', 'n6000.000', 'n6240.000', 'n5760.000', 'n5760.000', 'n5760.000', 'n5760.000', 'n5760.000'], '813': '814966', '839': '1204938', '888': ['172646 1154859', '172646 1154859', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646'], '917': 'n372361069212.000', '93': '769062', '981': '769062'}"
2,2,1

In [8]:
def combine_columns(row):
    return  (
        "ID: " + str(row.itemid) + "; " +
        "categoryid: " + str(row.categoryid) + "; " +
        "available: " + row.available + "; " +
        "properties: " + str(row.merged_properties) + "; " 
    )

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def get_embeddings(texts, model="text-embedding-ada-002"):
    return openai.Embedding.create(input=texts, model=model).data

def get_list_from_embeddings(embedding_result):
    list = []
    for entry in embedding_result:
        list.append(entry.embedding)
    return list

In [9]:
# pd.set_option('display.max_colwidth', 100000)
# average_length = product_data_embeddings['combined'].apply(len).max()
# average_length

In [10]:
product_embeddings_path = "openai/product_embeddings_retailrocket.pkl"
recreate_embeddings = False
if not os.path.exists(basepath + product_embeddings_path) or recreate_embeddings:
    product_data_embeddings = product_data[['itemid']].copy()
    product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
    # print(product_data_embeddings.combined.tolist())
    print('average length: ' + product_data_embeddings['combined'].apply(len).max())
    estimate_price(product_data_embeddings.shape[0])
    # embed each row separately
    # product_data_embeddings['embedding'] = product_data_embeddings.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
    # batch embed
    product_data_embeddings['embedding'] = get_list_from_embeddings(get_embeddings(product_data_embeddings.combined.tolist()))
    product_data_embeddings.to_pickle(basepath + product_embeddings_path)
else:
    product_data_embeddings = pd.read_pickle(basepath + product_embeddings_path)
display(product_data_embeddings)

itemid  \
0    309858   
1     89393   
2     88277   
3     90911   
4     72133   
..      ...   
995   79372   
996  155248   
997  165803   
998  443256   
999   35246   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [15]:
product_data_embeddings.iloc[0].combined

"ID: 309858; categoryid: 799; available: 0; properties: {'1066': 'n1260.000 424566', '112': '679677', '159': '519769', '202': '125782 n1440.000 822075', '227': '1308244', '283': '1308244 590597 125782 n1440.000 822075', '364': '909382', '384': '726612', '400': 'n586.800 424566', '566': 'n619.200 424566', '595': '873905', '6': '1308244', '678': '590597', '689': '150169 274770 79212', '698': '1308244', '764': '1285872', '776': '1081162', '790': 'n161400.000', '839': '590597', '853': '826838', '888': '125782 n1440.000 822075', '917': '1301157 n1440.000 822075', '934': '638115', '964': 'n2640.000 1128577', '992': 'n21.600 404373', '997': 'n360.000 863188'}; "

In [12]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    similarities = [cosine_similarity(query_embedding, neighbor) for neighbor in neighbors_embedding]
    indices = np.flip(np.argsort(similarities)[-n:])
    scores = [similarities[i] for i in indices]
    # Get the n neighbors with the highest similarity values
    n_neighbors = [neighbors_embedding[i] for i in indices]

    return indices, scores


indices, scores = get_n_nearest_neighbors(product_data_embeddings.embedding[0], product_data_embeddings.embedding)
display(indices, scores)

array([  0, 915, 716, 879, 504])

[0.9999999999999999,
 0.9869130461500556,
 0.9837275796996506,
 0.9835098388330239,
 0.9834647354448314]

In [13]:
product_data_embeddings.iloc[indices].combined

0                                                                                                                                                                                                                                               ID: 309858; categoryid: 799; available: 0; properties: {'1066': 'n1260.000 424566', '112': '679677', '159': '519769', '202': '125782 n1440.000 822075', '227': '1308244', '283': '1308244 590597 125782 n1440.000 822075', '364': '909382', '384': '726612', '400': 'n586.800 424566', '566': 'n619.200 424566', '595': '873905', '6': '1308244', '678': '590597', '689': '150169 274770 79212', '698': '1308244', '764': '1285872', '776': '1081162', '790': 'n161400.000', '839': '590597', '853': '826838', '888': '125782 n1440.000 822075', '917': '1301157 n1440.000 822075', '934': '638115', '964': 'n2640.000 1128577', '992': 'n21.600 404373', '997': 'n360.000 863188'}; 
915    ID: 419112; categoryid: 295; available: 0; properties: {'1066': 'n192.000 606827', '112': '

In [14]:
product_data.iloc[indices]

,itemid,categoryid,available,properties,property_values,merged_properties
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189, 202, 225, 227, 283, 364, 42, 561, 6, 678, 698, 764, 776, 790, 839, 869, 888, 917]","[1276750, n3.168 1144008, n15360.000 628176 n12288.000, 679677, 1168476, n4800.000 270060 924073, 519769, n96.000 1206660, 708480, 478989, 1301326, 1152934 1238769, 66094 372274 478989, 20716, n204.000, 1294803 101489, [66094, 66094, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769], 372274, 1152934 1238769, 1285872, 318611, n91200.000, 372274, 769062, 478989, 478989]","{'1036': '1276750', '1056': 'n3.168 1144008', '11': 'n15360.000 628176 n12288.000', '112': '679677', '127': '1168476', '139': 'n4800.000 270060 924073', '159': '519769', '177': 'n96.000 1206660', '189': '708480', '202': '478989', '225': '1301326', '227': '1152934 1238769', '283': '66094 372274 478989', '364': '20716', '42': 'n204.000', '561': '1294803 101489', '6': ['66094', '66094', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769'], '678': '372274', '698': '1152934 1238769', '764': '1285872', '776': '318611', '790': 'n91200.000', '839': '372274', '869': '769062', '888': '478989', '917': '478989'}"
915,1011,1135,1,"[1066, 112, 159, 202, 227, 230, 283, 348, 364, 400, 566, 591, 6, 62, 678, 698, 764, 776, 790, 839, 888, 899, 900, 917, 932]","[n714.000 424566, 679677, 519769, 1247872 210586, 430845 98427, 1096229, 980836 430845 98427 688107 1247872 210586, 530843, 568429, [n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n552.000 639502 n720.000 424566, n714.000 424566, n714.000 424566, n714.000 424566], n657.600 424566, 1116693, 430845 98427, 530843, 688107, 430845 98427, 1285872, 20285, [n376200.000, n376200.000, n367800.000, n337560.000, n331800.000, n325440.000, n322440.000, n322440.000, n322440.000, n322440.000, n322440.000, n328800.000, n314520.000, n312840.000, n307200.000, n326160.000, n355680.000, n355680.000], 688107, 1247872 210586, 980836, n714.000 424566, 1247872 210586, 980836]","{'1066': 'n714.000 424566', '112': '679677', '159': '519769', '202': '1247872 210586', '227': '430845 98427', '230': '1096229', '283': '980836 430845 98427 688107 1247872 210586', '348': '530843', '364': '568429', '400': ['n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n552.000 639502 n720.000 424566', 'n714.000 424566', 'n714.000 424566', 'n714.000 424566'], '566': 'n657.600 424566', '591': '1116693', '6': '430845 98427', '62': '530843', '678': '688107', '698': '430845 98427', '764': '1285872', '776': '20285', '790': ['n376200.000', 'n376200.000', 'n367800.000', 'n337560.000', 'n331800.000', 'n325440.000', 'n322440.000', 'n322440.000', 'n322440.000', 'n322440.000', 'n322440.000', 'n328800.000', 'n314520.000', 'n312840.000', 'n307200.000', 'n326160.000', 'n355680.000', 'n355680.000'], '839':